In [1]:
import cyclehunter

from cyclehunter import *

import numpy as np
from scipy.optimize import minimize
from scipy.linalg import eig
import matplotlib.pyplot as plt
import time

# PhiK Class

Most relevant methods are `generate_states`, `prime_orbits` and `hunt`.
(I need to change from orbits to cycles oops..)

Class instantiation expects $n$, $k$, and $\mu^2$.

In [2]:
n = 3
k = 3
musqr = 5
x = PhiK(n, k, musqr)


Relevant attributes are `n`, `k`, `musqr`, `states`. The `states` are empty until we populate them via `generate_states`

In [3]:
print(type(x.states))

<class 'NoneType'>


In [4]:
x = x.generate_states(prime=False)

In [5]:
print(type(x.states))

<class 'numpy.ndarray'>


Prime orbits are default, but can be set to False and then reproduced by calling `prime_orbits` function

In [6]:
x.states = x.prime_orbits()

In [7]:
x.states

array([[-1, -1,  0],
       [-1, -1,  1],
       [ 0, -1,  0],
       [ 0, -1,  1]], dtype=int32)

The numerical optimization routine is built into the class as opposed to be a separate module like it is in `orbithunter`;
this package's sister package.

In [8]:
converged_states = x.hunt()

Before after comparison

In [9]:
x.states 

array([[-1, -1,  0],
       [-1, -1,  1],
       [ 0, -1,  0],
       [ 0, -1,  1]], dtype=int32)

In [10]:
converged_states

array([[-1.06677935, -1.06677935, -0.33060481],
       [-1.19266329, -1.19266329,  1.32651357],
       [-0.1988372 , -1.15371662, -0.1988372 ],
       [ 0.        , -1.2649111 ,  1.2649111 ]])

Technically, I should be returning the optimization result to ensure convergence, however the two functions for the
equations of motion/cost function and the gradient of the cost function are: `eqn`, `cost`, `costgrad`.

In [11]:
x.states = converged_states

Cost is the $L_2$ norm squared of the governing equations `1/2 F^2`

In [12]:
x.cost()

1.9114600883812928e-12

Because it is the squared loss, we can expect the tolerance of each individual site in

In [13]:
x.eqn()

array([[ 2.25684180e-07,  2.25684180e-07, -1.77274262e-07],
       [ 7.06177658e-07,  7.06177658e-07,  1.38970459e-06],
       [-1.97346990e-07,  2.77151014e-08, -1.97346990e-07],
       [ 0.00000000e+00,  5.84086848e-07, -5.84086848e-07]])

Cost functional `1/2 F^2` is a scalar valued function; therefore the gradient is a vector, not a matrix.

In [14]:
x.costgrad()

array([-2.32111118e-06, -2.32111118e-06, -1.40164827e-06, -1.22201296e-05,
       -1.22201296e-05, -2.83650838e-05, -1.09476170e-06,  3.53418990e-08,
       -1.09476170e-06,  0.00000000e+00, -9.34539038e-06,  9.34539038e-06])

In [22]:
PhiK(n, k, musqr).generate_states(prime=True).states

array([], shape=(0, 1), dtype=int32)

In [26]:
# parameters
musqr = 5
n = 7
k = 3
for n in range(2, 10):
    t1 = time.time_ns()/10**9
    initial = PhiK(n, k, musqr).generate_states(prime=True)
    final = initial.hunt() 
    t2 = time.time_ns()/10**9
    print(f'Finding all prime {n}-cycles took {t2-t1} seconds')

Finding all prime 2-cycles took 0.002499103546142578 seconds
Finding all prime 3-cycles took 0.0020008087158203125 seconds
Finding all prime 4-cycles took 0.0059986114501953125 seconds
Finding all prime 5-cycles took 0.02649974822998047 seconds
Finding all prime 6-cycles took 0.20400023460388184 seconds
Finding all prime 7-cycles took 1.7719998359680176 seconds
Finding all prime 8-cycles took 15.42199993133545 seconds
Finding all prime 9-cycles took 139.9445059299469 seconds
